In [ ]:
import numpy as np
import torch
import torch.autograd as autograd   # computation graph
from torch.autograd import Variable
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import datetime, os
#hide tf logs 
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'} 
#0 (default) shows all, 1 to filter out INFO logs, 2 to additionally filter out WARNING logs, and 3 to additionally filter out ERROR logs
import scipy.optimize
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
#from scipy.fft import fft
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import os
from multi_gen_PINN import *
from scipy.stats import qmc

from torch.func import functional_call, vmap, vjp, jvp, jacrev

# generates same random numbers each time
np.random.seed(1234)

# Training of PINN Models (Poisson)

In [ ]:
N_Layers = 1
N_Neurons = 50
results_name = 'left_domain_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N_sin'
number_of_tests = 20
torch.set_default_tensor_type('torch.cuda.DoubleTensor')
layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
activation = torch.sin
N_f = 36 #Total number of collocation points 
lb = -np.pi # [-pi]
ub = -np.pi/3

opt = "Adam"
results_name = results_name + '_' + opt

if opt == "LM":
    num_epochs = 1000
    verbose = 100
elif opt == "Adam":
    num_epochs = 100000
    verbose = 10000

n_points = 151
x = np.linspace(-np.pi,np.pi,n_points)


def u_sol(x):
    f = 0
    for k in range(1,6):
        f += np.sin(2*k*x)/(2*k)
    return f
        
u = u_sol(x)

def u_de(x):
    
    f_prime = 0
    f_second = 0
    f_third = 0
    f_fourth = 0
    
    for k in range(1,6):
        
        f_prime += np.cos(2*k*x)
        f_second += -2*k*np.sin(2*k*x)
        f_third += -4*(k**2)*np.cos(2*k*x)
        f_fourth += 8*(k**3)*np.sin(2*k*x)
        
    return f_prime, f_second, f_third, f_fourth

u_prime, u_second, u_third, u_fourth = u_de(x)


###################
#### TEST DATA ####
###################

X_u_test = torch.tensor(x).view(n_points, 1)

u = torch.tensor(u).view(n_points, 1)



for i in range(number_of_tests):

    print('STARTING ITERATION ' + str(i) + '...')

    X_f_train, X_u_train, u_train = trainingdata(N_f, lb, ub, u_sol)
    X_f_train = torch.tensor(X_f_train)
    X_u_train = torch.tensor(X_u_train)
    u_train = torch.tensor(u_train)
    
    PINN = NN_Poisson(layers, activation)
    u_pred, u_pred_x, u_pred_xx, error, times = trainer(PINN, X_f_train, X_u_train, u_train, X_u_test, u_sol, opt, num_epochs=num_epochs, verbose = verbose)

    if i:

        all_u_pred = torch.cat((all_u_pred, u_pred), dim=1)
        all_u_pred_x = torch.cat((all_u_pred_x, u_pred_x), dim=1)
        all_u_pred_xx = torch.cat((all_u_pred_xx, u_pred_xx), dim=1)
    
        all_errors = torch.cat((all_errors, error), dim=1)
        all_times = torch.cat((all_times, times), dim=0)


    else:
        all_u_pred = u_pred
        all_u_pred_x = u_pred_x
        all_u_pred_xx = u_pred_xx

        all_errors = error
        all_times = times



create_directory('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name + '/MGP_all_U_errors_' + results_name, all_errors.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name + '/MGP_all_U_pred_' + results_name, all_u_pred.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name + '/MGP_all_U_pred_prime_' + results_name, all_u_pred_x.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name + '/MGP_all_U_pred_second_' + results_name, all_u_pred_xx.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name + '/MGP_all_U_times_' + results_name, all_times.detach().numpy(), delimiter=',')


In [ ]:
#N_Layers = 10
N_Neurons = 50
N_Layers = 1
number_of_tests = 20
torch.set_default_tensor_type('torch.cuda.DoubleTensor')
#activation = torch.nn.Tanh()
N_f = 36 #Total number of collocation points 
ub = -np.pi/3
lb = -np.pi # [-pi]

opt = "Adam"

if opt == "LM":
    num_epochs = 1000
    verbose = 500
elif opt == "Adam":
    num_epochs = 100000
    verbose = 50000

n_points = 151
x = np.linspace(-np.pi,np.pi,n_points)


def u_sol(x):
    f = 0
    for k in range(1,6):
        f += np.sin(2*k*x)/(2*k)
    return f
        
u = u_sol(x)

def u_de(x):
    
    f_prime = 0
    f_second = 0
    f_third = 0
    f_fourth = 0
    
    for k in range(1,6):
        
        f_prime += np.cos(2*k*x)
        f_second += -2*k*np.sin(2*k*x)
        f_third += -4*(k**2)*np.cos(2*k*x)
        f_fourth += 8*(k**3)*np.sin(2*k*x)
        
    return f_prime, f_second, f_third, f_fourth

u_prime, u_second, u_third, u_fourth = u_de(x)


###################
#### TEST DATA ####
###################

X_u_test = torch.tensor(x).view(n_points, 1)

u = torch.tensor(u).view(n_points, 1)
#domain_sizes = [np.pi/24, np.pi/12, np.pi/6, np.pi/3, 2*np.pi/3]
#which = ['_toosmall', '_smallest','_smaller', '_small',  '_full']
#N_fs = [3, 5, 9, 18, 36]
which = ['_tanh', '_relu','_sin', '_swish']
activations = [torch.nn.Tanh(), torch.nn.ReLU(), torch.sin, torch.nn.SiLU()]
'''
for j in range(len(activations)):
    
    layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
    print('Testing on ' + which[j] + ' domain size')
    results_name = 'left_domain_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N' + which[j]
    results_name = results_name + '_' + opt
    activation = activations[j]
    
    for i in range(number_of_tests):

        print('STARTING ITERATION ' + str(i) + '...')

        X_f_train, X_u_train, u_train = trainingdata(N_f, lb, ub, u_sol)
        X_f_train = torch.tensor(X_f_train)
        X_u_train = torch.tensor(X_u_train)
        u_train = torch.tensor(u_train)

        PINN = NN_Poisson(layers, activation)
        u_pred, u_pred_x, u_pred_xx, error, times = trainer(PINN, X_f_train, X_u_train, u_train, X_u_test, u_sol, opt, num_epochs=num_epochs, verbose = verbose)

        if i:

            all_u_pred = torch.cat((all_u_pred, u_pred), dim=1)
            all_u_pred_x = torch.cat((all_u_pred_x, u_pred_x), dim=1)
            all_u_pred_xx = torch.cat((all_u_pred_xx, u_pred_xx), dim=1)

            all_errors = torch.cat((all_errors, error), dim=1)
            all_times = torch.cat((all_times, times), dim=0)


        else:
            all_u_pred = u_pred
            all_u_pred_x = u_pred_x
            all_u_pred_xx = u_pred_xx

            all_errors = error
            all_times = times



    create_directory('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name + '/MGP_all_U_errors_' + results_name, all_errors.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name + '/MGP_all_U_pred_' + results_name, all_u_pred.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name + '/MGP_all_U_pred_prime_' + results_name, all_u_pred_x.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name + '/MGP_all_U_pred_second_' + results_name, all_u_pred_xx.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name + '/MGP_all_U_times_' + results_name, all_times.detach().numpy(), delimiter=',')


'''
opt = "LM"

if opt == "LM":
    num_epochs = 1000
    verbose = 500
elif opt == "Adam":
    num_epochs = 100000
    verbose = 50000
    
which = ['_relu','_sin', '_swish']
activations = [torch.nn.ReLU(), torch.sin, torch.nn.SiLU()]

for j in range(len(activations)):
    
    layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
    print('Testing on ' + which[j] + ' domain size')
    results_name = 'left_domain_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N' + which[j]
    results_name = results_name + '_' + opt
    activation = activations[j]
    
    for i in range(number_of_tests):

        print('STARTING ITERATION ' + str(i) + '...')

        X_f_train, X_u_train, u_train = trainingdata(N_f, lb, ub, u_sol)
        X_f_train = torch.tensor(X_f_train)
        X_u_train = torch.tensor(X_u_train)
        u_train = torch.tensor(u_train)

        PINN = NN_Poisson(layers, activation)
        u_pred, u_pred_x, u_pred_xx, error, times = trainer(PINN, X_f_train, X_u_train, u_train, X_u_test, u_sol, opt, num_epochs=num_epochs, verbose = verbose)

        if i:

            all_u_pred = torch.cat((all_u_pred, u_pred), dim=1)
            all_u_pred_x = torch.cat((all_u_pred_x, u_pred_x), dim=1)
            all_u_pred_xx = torch.cat((all_u_pred_xx, u_pred_xx), dim=1)

            all_errors = torch.cat((all_errors, error), dim=1)
            all_times = torch.cat((all_times, times), dim=0)


        else:
            all_u_pred = u_pred
            all_u_pred_x = u_pred_x
            all_u_pred_xx = u_pred_xx

            all_errors = error
            all_times = times



    create_directory('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name + '/MGP_all_U_errors_' + results_name, all_errors.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name + '/MGP_all_U_pred_' + results_name, all_u_pred.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name + '/MGP_all_U_pred_prime_' + results_name, all_u_pred_x.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name + '/MGP_all_U_pred_second_' + results_name, all_u_pred_xx.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name + '/MGP_all_U_times_' + results_name, all_times.detach().numpy(), delimiter=',')


# Training of PINN Models (Oscillator)

In [ ]:
import numpy as np
import torch
import torch.autograd as autograd   # computation graph
from torch.autograd import Variable
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import datetime, os
#hide tf logs 
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'} 
#0 (default) shows all, 1 to filter out INFO logs, 2 to additionally filter out WARNING logs, and 3 to additionally filter out ERROR logs
import scipy.optimize
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
#from scipy.fft import fft
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import os
from multi_gen_PINN import *
from scipy.stats import qmc

from torch.func import functional_call, vmap, vjp, jvp, jacrev

# generates same random numbers each time
np.random.seed(1234)

In [ ]:
N_Layers = 1
N_Neurons = 50
results_name = 'Oscillator_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N'
number_of_tests = 2
torch.set_default_tensor_type('torch.cuda.DoubleTensor')
layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
activation = torch.nn.Tanh()
N_f = 36 #Total number of collocation points 
lb = 0.0 # [-pi]
ub = 0.5

opt = "Adam"
results_name = results_name + '_' + opt

if opt == "LM":
    num_epochs = 1000
    verbose = 100
elif opt == "Adam":
    num_epochs = 100000
    verbose = 10000

n_points = 101
x = np.linspace(0.0,1.0,n_points)

d = 2.0
w0 = 20.0
mu = 2*d
k = w0**2

class oscillator(nn.Module): 
    def __init__(self, d, w0):
        super().__init__()

        self.d = d
        self.w0 = w0
        self.w = np.sqrt(self.w0**2-self.d**2)
        self.phi = np.arctan(-self.d/self.w)
        self.A = 1/(2*np.cos(self.phi))
    
    def forward(self, x):
        return 2*self.A*np.exp(-self.d*x)*np.cos(self.w*x + self.phi)
    
u_sol = oscillator(d, w0)
        
u = u_sol(x)
'''
def u_de(x):
    
    f_prime = 0
    f_second = 0
    f_third = 0
    f_fourth = 0
    
    for k in range(1,6):
        
        f_prime += np.cos(2*k*x)
        f_second += -2*k*np.sin(2*k*x)
        f_third += -4*(k**2)*np.cos(2*k*x)
        f_fourth += 8*(k**3)*np.sin(2*k*x)
        
    return f_prime, f_second, f_third, f_fourth

u_prime, u_second, u_third, u_fourth = u_de(x)
'''

###################
#### TEST DATA ####
###################

X_u_test = torch.tensor(x).view(n_points, 1)

u = torch.tensor(u).view(n_points, 1)



for i in range(number_of_tests):

    print('STARTING ITERATION ' + str(i) + '...')

    X_f_train, X_u_train, u_train = trainingdata(N_f, lb, ub, u_sol, dynamical_system = True)
    X_f_train = torch.tensor(X_f_train)
    X_u_train = torch.tensor(X_u_train)
    u_train = torch.tensor(u_train)
    
    PINN = NN_Oscillator(layers, activation, mu, k)
    u_pred, u_pred_x, u_pred_xx, error, times = trainer(PINN, X_f_train, X_u_train, u_train, X_u_test, u_sol, opt, num_epochs=num_epochs, verbose = verbose)

    if i:

        all_u_pred = torch.cat((all_u_pred, u_pred), dim=1)
        all_u_pred_x = torch.cat((all_u_pred_x, u_pred_x), dim=1)
        all_u_pred_xx = torch.cat((all_u_pred_xx, u_pred_xx), dim=1)
    
        all_errors = torch.cat((all_errors, error), dim=1)
        all_times = torch.cat((all_times, times), dim=0)

    else:
        all_u_pred = u_pred
        all_u_pred_x = u_pred_x
        all_u_pred_xx = u_pred_xx

        all_errors = error
        all_times = times


create_directory('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name + '/MGP_all_U_errors_' + results_name, all_errors.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name + '/MGP_all_U_pred_' + results_name, all_u_pred.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name + '/MGP_all_U_pred_prime_' + results_name, all_u_pred_x.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name + '/MGP_all_U_pred_second_' + results_name, all_u_pred_xx.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name + '/MGP_all_U_times_' + results_name, all_times.detach().numpy(), delimiter=',')

In [ ]:
N_Layers = 1
N_Neurons = 50
#results_name = 'Oscillator_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N'
number_of_tests = 20
torch.set_default_tensor_type('torch.cuda.DoubleTensor')
#layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
#activation = torch.nn.Tanh()


opt = "Adam"
#results_name = results_name + '_' + opt

if opt == "LM":
    num_epochs = 1000
    verbose = 100
elif opt == "Adam":
    num_epochs = 100000
    verbose = 10000

n_points = 101
x = np.linspace(0.0,1.0,n_points)

d = 2.0
w0 = 20.0
mu = 2*d
k = w0**2

class oscillator(nn.Module): 
    def __init__(self, d, w0):
        super().__init__()

        self.d = d
        self.w0 = w0
        self.w = np.sqrt(self.w0**2-self.d**2)
        self.phi = np.arctan(-self.d/self.w)
        self.A = 1/(2*np.cos(self.phi))
    
    def forward(self, x):
        return 2*self.A*np.exp(-self.d*x)*np.cos(self.w*x + self.phi)
    
u_sol = oscillator(d, w0)
        
u = u_sol(x)
'''
def u_de(x):
    
    f_prime = 0
    f_second = 0
    f_third = 0
    f_fourth = 0
    
    for k in range(1,6):
        
        f_prime += np.cos(2*k*x)
        f_second += -2*k*np.sin(2*k*x)
        f_third += -4*(k**2)*np.cos(2*k*x)
        f_fourth += 8*(k**3)*np.sin(2*k*x)
        
    return f_prime, f_second, f_third, f_fourth

u_prime, u_second, u_third, u_fourth = u_de(x)
'''

###################
#### TEST DATA ####
###################

X_u_test = torch.tensor(x).view(n_points, 1)

u = torch.tensor(u).view(n_points, 1)


opt = "Adam"

if opt == "LM":
    num_epochs = 1000
    verbose = 500
elif opt == "Adam":
    num_epochs = 100000
    verbose = 50000
    
    
#N_f = 36 #Total number of collocation points 
lb = 0.0 # [-pi]
ub = 0.5

#ubss = [0.5, 0.25, 0.125, 0.0625, 0.03125]
#which = ['full', 'small', 'smaller', 'smallest', 'toosmall']
N_f = 36
    
which = ['_tanh', '_relu','_sin', '_swish']
activations = [torch.nn.Tanh(), torch.nn.ReLU(), torch.sin, torch.nn.SiLU()]
#pointize = [9, 18, 25, 36, 50, 100]

for j in range(len(which)):
    
    layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
    print('Testing with ' + which[j] + ' activation')
    results_name = 'Oscillator_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N' + which[j]
    results_name = results_name + '_' + opt
    activation = activations[j]
    
    for i in range(number_of_tests):

        print('STARTING ITERATION ' + str(i) + '...')

        X_f_train, X_u_train, u_train = trainingdata(N_f, lb, ub, u_sol, dynamical_system = True)
        X_f_train = torch.tensor(X_f_train)
        X_u_train = torch.tensor(X_u_train)
        u_train = torch.tensor(u_train)

        PINN = NN_Oscillator(layers, activation, mu, k)
        u_pred, u_pred_x, u_pred_xx, error, times = trainer(PINN, X_f_train, X_u_train, u_train, X_u_test, u_sol, opt, num_epochs=num_epochs, verbose = verbose)

        if i:

            all_u_pred = torch.cat((all_u_pred, u_pred), dim=1)
            all_u_pred_x = torch.cat((all_u_pred_x, u_pred_x), dim=1)
            all_u_pred_xx = torch.cat((all_u_pred_xx, u_pred_xx), dim=1)

            all_errors = torch.cat((all_errors, error), dim=1)
            all_times = torch.cat((all_times, times), dim=0)

        else:
            all_u_pred = u_pred
            all_u_pred_x = u_pred_x
            all_u_pred_xx = u_pred_xx

            all_errors = error
            all_times = times


    create_directory('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name + '/MGP_all_U_errors_' + results_name, all_errors.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name + '/MGP_all_U_pred_' + results_name, all_u_pred.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name + '/MGP_all_U_pred_prime_' + results_name, all_u_pred_x.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name + '/MGP_all_U_pred_second_' + results_name, all_u_pred_xx.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name + '/MGP_all_U_times_' + results_name, all_times.detach().numpy(), delimiter=',')
    

    

opt = "LM"

if opt == "LM":
    num_epochs = 1000
    verbose = 500
elif opt == "Adam":
    num_epochs = 100000
    verbose = 50000
    
#N_f = 36 #Total number of collocation points 
lb = 0.0 # [-pi]
ub = 0.5

#ubss = [0.5, 0.25, 0.125, 0.0625, 0.03125]
#which = ['full', 'small', 'smaller', 'smallest', 'toosmall']
N_f = 36
    
which = ['_tanh', '_relu','_sin', '_swish']
activations = [torch.nn.Tanh(), torch.nn.ReLU(), torch.sin, torch.nn.SiLU()]
#pointize = [9, 18, 25, 36, 50, 100]

for j in range(len(which)):
    
    layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
    print('Testing with ' + which[j] + ' activation')
    results_name = 'Oscillator_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N' + which[j]
    results_name = results_name + '_' + opt
    activation = activations[j]
    
    for i in range(number_of_tests):

        print('STARTING ITERATION ' + str(i) + '...')

        X_f_train, X_u_train, u_train = trainingdata(N_f, lb, ub, u_sol, dynamical_system = True)
        X_f_train = torch.tensor(X_f_train)
        X_u_train = torch.tensor(X_u_train)
        u_train = torch.tensor(u_train)

        PINN = NN_Oscillator(layers, activation, mu, k)
        u_pred, u_pred_x, u_pred_xx, error, times = trainer(PINN, X_f_train, X_u_train, u_train, X_u_test, u_sol, opt, num_epochs=num_epochs, verbose = verbose)

        if i:

            all_u_pred = torch.cat((all_u_pred, u_pred), dim=1)
            all_u_pred_x = torch.cat((all_u_pred_x, u_pred_x), dim=1)
            all_u_pred_xx = torch.cat((all_u_pred_xx, u_pred_xx), dim=1)

            all_errors = torch.cat((all_errors, error), dim=1)
            all_times = torch.cat((all_times, times), dim=0)

        else:
            all_u_pred = u_pred
            all_u_pred_x = u_pred_x
            all_u_pred_xx = u_pred_xx

            all_errors = error
            all_times = times


    create_directory('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name + '/MGP_all_U_errors_' + results_name, all_errors.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name + '/MGP_all_U_pred_' + results_name, all_u_pred.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name + '/MGP_all_U_pred_prime_' + results_name, all_u_pred_x.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name + '/MGP_all_U_pred_second_' + results_name, all_u_pred_xx.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name + '/MGP_all_U_times_' + results_name, all_times.detach().numpy(), delimiter=',')
    

    


# Training of PINN Models (Nonlinear)

In [2]:
import numpy as np
import torch
import torch.autograd as autograd   # computation graph
from torch.autograd import Variable
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import datetime, os
#hide tf logs 
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'} 
#0 (default) shows all, 1 to filter out INFO logs, 2 to additionally filter out WARNING logs, and 3 to additionally filter out ERROR logs
import scipy.optimize
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
#from scipy.fft import fft
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import os
from multi_gen_PINN import *
from scipy.stats import qmc
import scipy.special as ss

from torch.func import functional_call, vmap, vjp, jvp, jacrev

# generates same random numbers each time
np.random.seed(1234)

In [3]:
N_Layers = 1
N_Neurons = 50
results_name = 'Nonlinear_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N_sin'
number_of_tests = 20
torch.set_default_tensor_type('torch.cuda.DoubleTensor')
layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
activation = torch.sin
N_f = 36 #Total number of collocation points 
lb = 0.0 # [-pi]
ub = np.pi

opt = "LM"
results_name = results_name + '_' + opt

if opt == "LM":
    num_epochs = 1000
    verbose = 200
elif opt == "Adam":
    num_epochs = 100000
    verbose = 10000

n_points = 201
x = np.linspace(0.0,2*np.pi,n_points)


def u_sol(x):
    _, cn, dn, _ = ss.ellipj(np.sqrt(10)*x, 0.5)
    cd = cn/dn
    return 2*np.arcsin(cd/np.sqrt(2.0))
     
u = u_sol(x)

def u_de(x):
    f = 2*np.pi
    return f*np.cos(f*x), -(f**2)*np.sin(f*x), -(f**3)*np.cos(f*x), (f**4)*np.sin(f*x)

u_prime, u_second, u_third, u_fourth = u_de(x)


###################
#### TEST DATA ####
###################

X_u_test = torch.tensor(x).view(n_points, 1)

u = torch.tensor(u).view(n_points, 1)



for i in range(number_of_tests):

    print('STARTING ITERATION ' + str(i) + '...')

    X_f_train, X_u_train, u_train = trainingdata(N_f, lb, ub, u_sol, dynamical_system=True)
    X_f_train = torch.tensor(X_f_train)
    X_u_train = torch.tensor(X_u_train)
    u_train = torch.tensor(u_train)
    
    PINN = NN_Nonlinear(layers, activation)
    u_pred, u_pred_x, u_pred_xx, error, times = trainer(PINN, X_f_train, X_u_train, u_train, X_u_test, u_sol, opt, num_epochs=num_epochs, verbose = verbose)

    if i:

        all_u_pred = torch.cat((all_u_pred, u_pred), dim=1)
        all_u_pred_x = torch.cat((all_u_pred_x, u_pred_x), dim=1)
        all_u_pred_xx = torch.cat((all_u_pred_xx, u_pred_xx), dim=1)
    
        all_errors = torch.cat((all_errors, error), dim=1)
        all_times = torch.cat((all_times, times), dim=0)


    else:
        all_u_pred = u_pred
        all_u_pred_x = u_pred_x
        all_u_pred_xx = u_pred_xx

        all_errors = error
        all_times = times



create_directory('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name + '/MGP_all_U_errors_' + results_name, all_errors.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name + '/MGP_all_U_pred_' + results_name, all_u_pred.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name + '/MGP_all_U_pred_prime_' + results_name, all_u_pred_x.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name + '/MGP_all_U_pred_second_' + results_name, all_u_pred_xx.detach().numpy(), delimiter=',')

create_directory('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name)
np.savetxt('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name + '/MGP_all_U_times_' + results_name, all_times.detach().numpy(), delimiter=',')


STARTING ITERATION 0...
Epoch 0, Loss function (total, residuals, boundaries):
tensor(1.4926)
Epoch 200, Loss function (total, residuals, boundaries):
tensor(0.1467)
Epoch 400, Loss function (total, residuals, boundaries):
tensor(0.1451)
Epoch 600, Loss function (total, residuals, boundaries):
tensor(0.1442)


KeyboardInterrupt: 

In [ ]:
number_of_tests = 20
torch.set_default_tensor_type('torch.cuda.DoubleTensor')
'''
opt = "Adam"


if opt == "LM":
    num_epochs = 1000
    verbose = 500
elif opt == "Adam":
    num_epochs = 100000
    verbose = 50000

n_points = 201
x = np.linspace(0.0,2*np.pi,n_points)

def u_sol(x):
    _, cn, dn, _ = ss.ellipj(np.sqrt(10)*x, 0.5)
    cd = cn/dn
    return 2*np.arcsin(cd/np.sqrt(2.0))
     
u = u_sol(x)

def u_de(x):
    f = 2*np.pi
    return f*np.cos(f*x), -(f**2)*np.sin(f*x), -(f**3)*np.cos(f*x), (f**4)*np.sin(f*x)

u_prime, u_second, u_third, u_fourth = u_de(x)


###################
#### TEST DATA ####
###################

X_u_test = torch.tensor(x).view(n_points, 1)

u = torch.tensor(u).view(n_points, 1)

#neuronize = [10, 20, 50, 100, 200]
#layerize = [10]
activize = [torch.nn.Tanh(), torch.sin, torch.nn.ReLU(), torch.nn.SiLU()]
which = ['tanh', 'sin', 'relu', 'swish']

for j in range(len(activize)):
    
    N_Layers = 1
    N_Neurons = 50
    results_name = 'Nonlinear_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N_' + which[j]
    print('Testing with ' + which[j] + ' activation')

    layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
    activation = activize[j]
    N_f = 36 #Total number of collocation points 
    lb = 0.0 # [-pi]
    ub = np.pi

    results_name = results_name + '_' + opt


    for i in range(number_of_tests):

        print('STARTING ITERATION ' + str(i) + '...')

        X_f_train, X_u_train, u_train = trainingdata(N_f, lb, ub, u_sol, dynamical_system=True)
        X_f_train = torch.tensor(X_f_train)
        X_u_train = torch.tensor(X_u_train)
        u_train = torch.tensor(u_train)

        PINN = NN_Nonlinear(layers, activation)
        u_pred, u_pred_x, u_pred_xx, error, times = trainer(PINN, X_f_train, X_u_train, u_train, X_u_test, u_sol, opt, num_epochs=num_epochs, verbose = verbose)

        if i:

            all_u_pred = torch.cat((all_u_pred, u_pred), dim=1)
            all_u_pred_x = torch.cat((all_u_pred_x, u_pred_x), dim=1)
            all_u_pred_xx = torch.cat((all_u_pred_xx, u_pred_xx), dim=1)

            all_errors = torch.cat((all_errors, error), dim=1)
            all_times = torch.cat((all_times, times), dim=0)


        else:
            all_u_pred = u_pred
            all_u_pred_x = u_pred_x
            all_u_pred_xx = u_pred_xx

            all_errors = error
            all_times = times



    create_directory('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name + '/MGP_all_U_errors_' + results_name, all_errors.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name + '/MGP_all_U_pred_' + results_name, all_u_pred.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name + '/MGP_all_U_pred_prime_' + results_name, all_u_pred_x.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name + '/MGP_all_U_pred_second_' + results_name, all_u_pred_xx.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name + '/MGP_all_U_times_' + results_name, all_times.detach().numpy(), delimiter=',')

    
'''
    
opt = "LM"


if opt == "LM":
    num_epochs = 1000
    verbose = 500
elif opt == "Adam":
    num_epochs = 100000
    verbose = 5000

n_points = 201
x = np.linspace(0.0,2*np.pi,n_points)


def u_sol(x):
    _, cn, dn, _ = ss.ellipj(np.sqrt(10)*x, 0.5)
    cd = cn/dn
    return 2*np.arcsin(cd/np.sqrt(2.0))
     
u = u_sol(x)

def u_de(x):
    f = 2*np.pi
    return f*np.cos(f*x), -(f**2)*np.sin(f*x), -(f**3)*np.cos(f*x), (f**4)*np.sin(f*x)

u_prime, u_second, u_third, u_fourth = u_de(x)


###################
#### TEST DATA ####
###################

X_u_test = torch.tensor(x).view(n_points, 1)

u = torch.tensor(u).view(n_points, 1)

#neuronize = [10, 20, 50, 100, 200]
#layerize = [10]
activize = [torch.nn.ReLU(), torch.nn.SiLU()]
which = ['relu', 'swish']

for j in range(len(activize)):
    
    N_Layers = 1
    N_Neurons = 50
    results_name = 'Nonlinear_' + str(N_Layers) + 'L_' + str(N_Neurons) + 'N_' + which[j]
    print('Testing with ' + which[j] + ' activation')

    layers = np.array([1] + N_Layers*[N_Neurons] + [1]) #2 hidden standard layers
    activation = activize[j]
    N_f = 36 #Total number of collocation points 
    lb = 0.0 # [-pi]
    ub = np.pi

    results_name = results_name + '_' + opt


    for i in range(number_of_tests):

        print('STARTING ITERATION ' + str(i) + '...')

        X_f_train, X_u_train, u_train = trainingdata(N_f, lb, ub, u_sol, dynamical_system=True)
        X_f_train = torch.tensor(X_f_train)
        X_u_train = torch.tensor(X_u_train)
        u_train = torch.tensor(u_train)

        PINN = NN_Nonlinear(layers, activation)
        u_pred, u_pred_x, u_pred_xx, error, times = trainer(PINN, X_f_train, X_u_train, u_train, X_u_test, u_sol, opt, num_epochs=num_epochs, verbose = verbose)

        if i:

            all_u_pred = torch.cat((all_u_pred, u_pred), dim=1)
            all_u_pred_x = torch.cat((all_u_pred_x, u_pred_x), dim=1)
            all_u_pred_xx = torch.cat((all_u_pred_xx, u_pred_xx), dim=1)

            all_errors = torch.cat((all_errors, error), dim=1)
            all_times = torch.cat((all_times, times), dim=0)


        else:
            all_u_pred = u_pred
            all_u_pred_x = u_pred_x
            all_u_pred_xx = u_pred_xx

            all_errors = error
            all_times = times



    create_directory('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_errors_' + results_name + '/MGP_all_U_errors_' + results_name, all_errors.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_' + results_name + '/MGP_all_U_pred_' + results_name, all_u_pred.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_prime_' + results_name + '/MGP_all_U_pred_prime_' + results_name, all_u_pred_x.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_pred_second_' + results_name + '/MGP_all_U_pred_second_' + results_name, all_u_pred_xx.detach().numpy(), delimiter=',')

    create_directory('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name)
    np.savetxt('Results/Multi_GEN_Results/MGP_all_U_times_' + results_name + '/MGP_all_U_times_' + results_name, all_times.detach().numpy(), delimiter=',')


Testing with relu activation
STARTING ITERATION 0...
Epoch 0, Loss function (total, residuals, boundaries):
tensor(7.5167)
Epoch 500, Loss function (total, residuals, boundaries):
tensor(1.1396)
Training time: 46.36
STARTING ITERATION 1...
Epoch 0, Loss function (total, residuals, boundaries):
tensor(1.3569)
Epoch 500, Loss function (total, residuals, boundaries):
tensor(1.1389)
Training time: 32.59
STARTING ITERATION 2...
Epoch 0, Loss function (total, residuals, boundaries):
tensor(15.2750)
Epoch 500, Loss function (total, residuals, boundaries):
tensor(1.1402)
Training time: 29.52
STARTING ITERATION 3...
Epoch 0, Loss function (total, residuals, boundaries):
tensor(3.5852)
Epoch 500, Loss function (total, residuals, boundaries):
tensor(1.1442)
Training time: 28.11
STARTING ITERATION 4...
Epoch 0, Loss function (total, residuals, boundaries):
tensor(2.9739)
Epoch 500, Loss function (total, residuals, boundaries):
tensor(1.1354)
Epoch 500, Loss function (total, residuals, boundaries):